<a href="https://colab.research.google.com/github/SLCFLAB/Data-Science-Python/blob/main/04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 투자지표 수집

In [24]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
!pip install finance-datareader
import FinanceDataReader as fdr
import numpy as np

In [4]:
stocks = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 전체
stocks.dropna(inplace = True) # 지표가 없는 선물 등은 홈페이지, 지역, 산업 등이 모두 결측으로 되어 있으므로 삭제하면 됨
base_url = "https://search.itooza.com/search.htm?seName={}&jl=&search_ck=&sm=&sd=2021-08-09&ed=2021-09-08&ds_de=&page=&cpv="

In [6]:
for code, name in stocks[:20][['Symbol', 'Name']].values:
    print(name)
    while True:
        try:
            url = base_url.format(code)
            html = requests.get(url, headers={'User-Agent':'Gils'}).content
            soup = BeautifulSoup(html, 'lxml')
            break
        except:
            time.sleep(15 * 60)
    
    # 만약 해당 종목 자료가 없다면, 아래 테이블이 없음
    table = soup.find("table", {"class":"ex"}) # class가 ex인 테이블을 찾음
    
    if table == None: # class가 ex인 테이블이 없으면 크롤링하지 않음
        continue

    idx = ["2021-2Q", "2021-1Q",
           "2020-4Q", "2020-3Q", "2020-2Q", "2020-1Q",
           "2019-4Q", "2019-3Q", "2019-2Q", "2019-1Q",
           "2018-4Q", "2018-3Q"] # 인덱스를 직접 정의

    table_body = table.find('tbody')  # table에서 tbody만 가져옴
    tr_list = table_body.find_all('tr') # tbody에서 행을 다 찾아서 저장

    result_dict = dict()
    for tr in tr_list:
        col_name = tr.find('th').text # 투자 지표 이름
        values = [td.text for td in tr.find_all('td')] # tr에 있는 모든 td의 text만 가져와서 저장
        result_dict[col_name] = values # 사전으로 저장

    result = pd.DataFrame.from_dict(result_dict)
    result.drop('주가', axis = 1, inplace = True) # 주가열도 필요없으니 삭제
    result['분기'] = idx
    
    result.to_csv(name + ".csv", index = False, encoding = "cp949")
    
    time.sleep(1) # 1초씩 재우기

3S
AJ네트웍스
AK홀딩스
APS홀딩스
AP시스템
AP위성
BGF
BGF리테일
BNGT
BNK금융지주
BYC
CBI
CJ
CJ CGV
CJ ENM
CJ 바이오사이언스
CJ대한통운
CJ씨푸드
CJ제일제당
CJ프레시웨이


# 한경컨센서스 리포트 수집

In [12]:
import requests
import re

In [8]:
stocks = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 전체

In [9]:
stocks['Symbol'] = stocks['Symbol'].astype(str)
def remove_noise_and_split_title(title):
    in_code = ''
    in_name = ''
    
    for code, name in stocks[['Symbol', 'Name']].values:
        if code in title and name in title:
            in_code = code
            in_name = name
    
    # 한글, 영어, 숫자 외 노이즈 제거
    clean_title = re.sub('[^A-Za-z0-9가-힣]', ' ', title)
    
    # 기업명 코드 수정
    clean_title = clean_title.replace(in_code, ' ')
    clean_title = clean_title.replace(in_name, ' ')
    while ' ' * 2 in clean_title:
        clean_title = clean_title.replace(' ' * 2, ' ')
    
    if in_name == '': # 기업명이 없는 제목이라면, 데이터에 추가하지 않음
        return [None] 
    else:
        return [in_name, in_code, clean_title]  

In [14]:
base_url = "http://consensus.hankyung.com/apps.analysis/analysis.list?&sdate=2016-09-01&edate=2021-08-31&report_type=CO&order_type=&now_page={}"
data = [] 

for page_no in range(1, 10):
    while True:
        try:
            url = base_url.format(page_no)
            html = requests.get(url, headers={'User-Agent':'Gils'}).content
            soup = BeautifulSoup(html, 'lxml')
            print("{}/{}".format(page_no, 4284))
            break
        except:
            time.sleep(15 * 60)

    table = soup.find("div", {"class":"table_style01"}).find('table')
    for tr in table.find_all("tr")[1:]: # 1번째 행부터 순회
        record = []
        for i, td in enumerate(tr.find_all("td")[:6]): # 6번째 셀까지 순회
            if i == 1:
                record += remove_noise_and_split_title(td.text) # remove_noise_title의 출력과 이어 붙임
            elif i == 3: # 노이즈가 껴있는 세번째 셀만 따로 처리
                record.append(td.text.replace(" ", "").replace("\r","").replace("\n",""))
            else: # 1번째 셀이 아니면:
                record.append(td.text) # 셀의 텍스트 값 그대로 입력
            
        if None not in record: # 레코드에 None이 없으면
            data.append(record)

    time.sleep(1) # 연결 끊김 방지를 위해, 1초씩 재움
    
data = pd.DataFrame(data, columns = ["작성일", "종목명", "종목코드", "제목", "적정가격", "평가의견", "작성자", "작성기관"])
data.to_csv("리포트_데이터.csv", index = False, encoding = "cp949")

1/4284
2/4284
3/4284
4/4284
5/4284
6/4284
7/4284
8/4284
9/4284


# OpenDart 활용

In [15]:
!pip install opendartreader

In [16]:
import OpenDartReader

API 키 발급 링크 : https://opendart.fss.or.kr/uss/umt/EgovMberInsertView.do

In [17]:
my_api = "7c21274166c816929c5a5c6dacffaa8315b7daeb"
dart = OpenDartReader(my_api)

In [18]:
display(dart.report("SK하이닉스","배당",2020,"11011"))

,rcept_no,corp_cls,corp_code,corp_name,se,thstrm,frmtrm,lwfr
0,20210330000776,Y,00164779,SK하이닉스,주당액면가액(원),"5,000","5,000","5,000"
1,20210330000776,Y,00164779,SK하이닉스,(연결)당기순이익(백만원),"4,755,102","2,005,975","15,540,111"
2,20210330000776,Y,00164779,SK하이닉스,(별도)당기순이익(백만원),"4,217,841","1,476,981","15,407,086"
3,20210330000776,Y,00164779,SK하이닉스,(연결)주당순이익(원),"6,952","2,933","22,255"
4,20210330000776,Y,00164779,SK하이닉스,현금배당금총액(백만원),"800,282","684,002","1,026,003"
5,20210330000776,Y,00164779,SK하이닉스,주식배당금총액(백만원),-,-,-
6,20210330000776,Y,00164779,SK하이닉스,(연결)현금배당성향(%),16.80,34.10,6.60
7,20210330000776,Y,00164779,SK하이닉스,현금배당수익률(%),1.00,1.10,2.50
8,20210330000776,Y,00164779,SK하이닉스,현금배당수익률(%),-,-,-
9,20210330000776,Y,00164779,SK하이닉스,주식배당수익률(%),-,-,-


In [19]:
stock_list = fdr.StockListing("KRX")

In [20]:
stock_list = stock_list.loc[stock_list['Market'].isin(["KOSPI","KOSDAQ"])]
stock_name_list = stock_list.loc[stock_list['Region'].notnull(),'Name'].tolist()

In [34]:
def find_dividends(stock_name, year):
  stock_name_report = dart.report(stock_name, "배당", year, "11011")
  if stock_name_report is None:
    return np.nan, np.nan, np.nan
  else:
    
    stock_name_report = stock_name_report.loc[stock_name_report['se']=="주당 현금배당금(원)"].iloc[0]
    
    thstrm_dividends = int(stock_name_report['thstrm'].replace('-','0').replace(',',''))
    frmtrm_dividends = int(stock_name_report['frmtrm'].replace('-','0').replace(',',''))
    lwfr_dividends = int(stock_name_report['lwfr'].replace('-','0').replace(',',''))
    
    return lwfr_dividends, frmtrm_dividends, thstrm_dividends

In [36]:
data = []
for idx, stock_name in enumerate(stock_name_list[:20]):
  print(idx+1, "/", len(stock_name_list))
  record = [stock_name]
  for year in [2015,2018,2020]:
    lwfr_dividends, frmtrm_dividends, thstrm_dividends = find_dividends(stock_name, year)

    if year != 2020:
      record += [lwfr_dividends, frmtrm_dividends, thstrm_dividends]
    else:
      record += [frmtrm_dividends, thstrm_dividends]
    
    data.append(record)
    time.sleep(0.5)

1 / 2370
2 / 2370
3 / 2370
4 / 2370
5 / 2370
6 / 2370
7 / 2370
8 / 2370
9 / 2370
10 / 2370
11 / 2370
12 / 2370
13 / 2370
14 / 2370
15 / 2370
16 / 2370
17 / 2370
18 / 2370
19 / 2370
20 / 2370


In [39]:
data = pd.DataFrame(data,columns=["stock_name","2015","2016","2017","2018","2019","2020","2021","2022"])
data.to_csv("배당목록.csv")